In [8]:
import requests
import mysql.connector
from mysql.connector import errorcode
import time
import discord_notifier


# Function to insert match IDs into the database in bulk
def insert_match_ids(match_ids):
    try:
        cnx = mysql.connector.connect(user='lol_user', password='your_password',
                                      host='35.203.46.85', database='lol_retention')
        cursor = cnx.cursor()

        # Preparing the bulk insert query
        add_match_ids = """
        INSERT INTO match_ids (match_id, puuid)
        VALUES (%s, %s)
        ON DUPLICATE KEY UPDATE
        match_id = VALUES(match_id), puuid = VALUES(puuid)
        """
        cursor.executemany(add_match_ids, [(match_id,) for match_id in match_ids])
        
        cnx.commit()
        cursor.close()
        cnx.close()
    except mysql.connector.Error as err:
        print("Error:", err)

# Function to pull match IDs from the API and insert them into the database
def pull_match_ids():
    api_key = "RGAPI-39403bc3-0460-4e9a-b6eb-ef92e74fabd0"
    base_url = "https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/"
    
    try:
        cnx = mysql.connector.connect(user='lol_user', password='your_password',
                                      host='35.203.46.85', database='lol_retention')
        cursor = cnx.cursor()

        ''' 
        ############################################################################
        #THIS CODE YOU RUN IT FOR WHEN YOU WANT TO INCREASE THE POOL OF MATCHES PER PLAYER
        # Fetching all PUUIDs from the summoner_details table
        query = "SELECT puuid FROM summoner_details"
        ############################################################################
        '''

        ###########################################################################
        #THIS SHOULD BE USE WHEN INCREASING THE PLAYERS POOL
        # Fetching all PUUIDs from summoner_details that do not have match details in match_details
        query = """
        SELECT sd.puuid 
        FROM summoner_details sd 
        LEFT JOIN match_details md ON sd.puuid = md.puuid 
        WHERE md.puuid IS NULL
        """
        ############################################################################
        
        cursor.execute(query)
        puuids = cursor.fetchall()
        print(f"{len(puuids)} PUUIDs retrieved for fetching match IDs.")


        
        all_match_id_puuid_pairs = set()  # Using a set to ensure uniqueness
        val = 1
        total = len(puuids)
        for (puuid,) in puuids:
            url = f"{base_url}{puuid}/ids?start=0&count=1&api_key={api_key}"  # Adjust 'count' as needed
            print(f"Fetching element: {val} our of {total}")
            response = requests.get(url)
            if response.status_code != 200:
                print("Error with the request:", response.status_code)
                continue
            match_ids = response.json()
            for match_id in match_ids:
                all_match_id_puuid_pairs.add((match_id, puuid))
            time.sleep(1.3)  # To respect the rate limit
            val += 1
        insert_match_ids(list(all_match_id_puuid_pairs))
    except mysql.connector.Error as err:
        print("Database error:", err)
    finally:
        if cnx.is_connected():
            cnx.close()

if __name__ == "__main__":
    try:
        pull_match_ids()
    except Exception as e:
        print(f"An error occurred: {e}")
    finally: 
        discord_notifier.send_discord_message()
    


7997 PUUIDs retrieved for fetching match IDs.
Fetching element: 1 our of 7997
Fetching element: 2 our of 7997
Fetching element: 3 our of 7997
Fetching element: 4 our of 7997
Fetching element: 5 our of 7997
Fetching element: 6 our of 7997
Fetching element: 7 our of 7997
Fetching element: 8 our of 7997
Fetching element: 9 our of 7997
Fetching element: 10 our of 7997
Fetching element: 11 our of 7997
Fetching element: 12 our of 7997
Fetching element: 13 our of 7997
Fetching element: 14 our of 7997
Fetching element: 15 our of 7997
Fetching element: 16 our of 7997
Fetching element: 17 our of 7997
Fetching element: 18 our of 7997
Fetching element: 19 our of 7997
Fetching element: 20 our of 7997
Fetching element: 21 our of 7997
Fetching element: 22 our of 7997
Fetching element: 23 our of 7997
Fetching element: 24 our of 7997
Fetching element: 25 our of 7997
Fetching element: 26 our of 7997
Fetching element: 27 our of 7997
Fetching element: 28 our of 7997
Fetching element: 29 our of 7997
Fetchi

KeyboardInterrupt: 